importing packages

In [2]:
!pip install langchain_core langchain_chroma langchain_text_splitters langchain langchain_google_genai sentence_transformers langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 10.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19

In [3]:
import pandas as pd
import numpy as np
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.documents import Document
from kaggle_secrets import UserSecretsClient
from langchain.load import dumps, loads

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
user_secrets = UserSecretsClient()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = user_secrets.get_secret("LANGCHAIN_KEY")

In [5]:
data_directory='/kaggle/input/sanad-dataset'
data=[]
for folder in os.listdir(data_directory):
    for file in os.listdir(f'{data_directory}/{folder}'):

            file = open(f"{data_directory}/{folder}/{file}", "r",encoding="utf8")
            content = file.read()

            data.append({'category': folder, 'Content': content})

            file.close()           

In [6]:
data[0]

{'category': 'Finance',
 'Content': 'دبي - "الخليج":حصدت شعاع كابيتال جائزة "أفضل شركة لإدارة الأصول في الإمارات" خلال حفل توزيع جوائز الأعمال المصرفية في الشرق الأوسط لعام 2014 الذي تنظمه مجلة إيميا فايننس . وتم اختيار شعاع لنيل الجائزة للسنة الخامسة على التوالي، تقديراً للأداء القوي الذي يحققه فريقها لإدارة الأصول .وقال الشيخ مكتوم بن حشر آل مكتوم، رئيس مجلس الإدارة التنفيذي لشعاع كابيتال: "نحن فخورون بالتكريم الذي حصل عليه فريقنا لإدارة الأصول مرّة جديدة، وذلك تقديراً لتميّزه في تحقيق مستوى متسق من العائدات لصالح العملاء . ويعدّ الفوز بهذه الجائزة لخمس مرّات متتالية إنجازاً كبيراً، ودافعاً قوياً لفريق إدارة الأصول ليواصلوا تحقيق هذا الأداء المتميّز" .وبلغ العائد الذي حققه الصندوقان الرئيسيان لدى شعاع لإدارة الأصول، وهما صندوق البوابة العربية وصندوق بوابة الإمارات، 2 .16% و2 .30% على التوالي خلال الفترة الماضية من العام 2014 . وتمتلك صناديق شعاع، التي أطلقت في العام ،1999 أطول سجل استثماري في المنطقة وقد حصل فريق إدارة الأصول لدى الشركة على مجموعة كبيرة من الجوائز خلال السنوات الأخير

In [7]:
documents = [Document(page_content=text["Content"]) for text in data[:40000]]

splitting data into documents

In [8]:
len(documents)

40000

Embedding to find similarity between texts and find relevant 

In [9]:
class embedding:
    def __init__(self):
        self.model = SentenceTransformer('all-mpnet-base-v2')
    def embed_documents(self,docs):
        embeddings= self.model.encode(docs)
        return embeddings.tolist()
    def embed_query(self,query):
        return self.model.encode(query).tolist()

In [10]:
embed_model=embedding()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
result=embed_model.embed_query("ما الجائزة التي حصدتها شعاع كابيتال ؟")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
len(result)

768

In [13]:
vector_database=Chroma.from_documents(documents=documents,embedding=embed_model)

Batches:   0%|          | 0/1250 [00:00<?, ?it/s]

In [14]:
retriever=vector_database.as_retriever(search_type="similarity",search_kwargs={'k':3})

In [15]:
retrieved_docs=retriever.invoke("لماذا تأجل اجتماع الجمعية العمومية غير العادية ل الدار العقارية ؟")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
retrieved_docs[0].page_content

'ينظم النادي الثقافي العربي أمسية للشاعر حسن عبدالسلام أبودية عند الثامنة من مساء يوم غد في مقر النادي في الشارقة .'

In [17]:
retrieved_docs[1].page_content

'أبلغ بنك دبي الإسلامي سوق دبي المالي بأن اجتماع الجمعية العمومية غير العادية للبنك أمس الأول قد تأجل نظراً لعدم اكتمال النصاب القانوني .'

In [18]:
retrieved_docs[2].page_content

'دبي الخليج: أعلن بنك المشرق أن الجمعية العمومية غير العادية وافقت على الزيادة المقترحة لسقف برنامج سندات القرض متوسطة الأجل باليورو من ملياري دولار إلى خمسة مليارات دولار .'

construct template

In [19]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
google_api_key = user_secrets.get_secret("GOOGLE_AI_STUDIO2")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",google_api_key=google_api_key,temperature=0)

In [63]:

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five arabic 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines without any additional thoughts outside the answers. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives 
    | llm 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)


In [64]:


def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
retrieval_chain = generate_queries | retriever.map() | get_unique_union




In [65]:


# RAG
# template = """Answer the following question based on this context:

# {context}

# Question: {question}
# """

template="""
you are an AI powered QA Assistant to provide accurate, contextually relevant answers to customer questions.
at the end of the answer you have to thank the user. 
the answer in arabic and in details.
you should answer based on the context provided:
{context}

what to do if answer is not included in the prompt for context:

1.you should appoligize to the user and say that you dont have the answer in your informations

for answer:
1. the output must be the answer only without any additional thoughts

Question: {question}

answer:
"""

prompt = ChatPromptTemplate.from_template(template)


final_rag_chain = (
    {"context": retrieval_chain, 
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

# final_rag_chain.invoke({"question":question})



In [67]:
answer=final_rag_chain.invoke("ما الحدث الذي ياخذ مكانا ف الشارقة؟" )
print(answer)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ينظم النادي الثقافي العربي أمسية للشاعر حسن عبدالسلام أبودية عند الثامنة من مساء يوم غد في مقر النادي في الشارقة. 

وينظم بيت الشعر في الشارقة أمسية للشاعر عبدالعزيز إسماعيل عبدالهادي ويقدم الأمسية ويديرها الدكتور بهجت الحديثي وذلك في الثامنة من مساء اليوم في مقر بيت الشعر في الشارقة القديمة. 

شكراً لك. 

